In [33]:
import sympy as sp
import numpy as np
from docplex.mp.model import Model
from quaos.hamiltonian import random_pauli_hamiltonian
from quaos.paulis.utils import find_symplectic_maps
from pyscipopt import Model as scip_model, quicksum
import numpy as np
from time import time


In [ ]:
def solve_HT_equals_PH(H, d):
    n, k = H.shape  # H is n×k

    mdl = Model("HT_equals_PH")

    # Unknown matrix M ∈ ℤ_d^{k×k}
    M = [[mdl.integer_var(lb=0, ub=d - 1, name=f"M_{i}_{j}") for j in range(k)] for i in range(k)]

    # Permutation matrix P ∈ {0,1}^{n×n}
    P = [[mdl.binary_var(name=f"P_{i}_{j}") for j in range(n)] for i in range(n)]

    # Enforce P to be a permutation matrix
    for i in range(n):
        mdl.add_constraint(mdl.sum(P[i][j] for j in range(n)) == 1)  # each row: 1 entry is 1
    for j in range(n):
        mdl.add_constraint(mdl.sum(P[i][j] for i in range(n)) == 1)  # each col: 1 entry is 1

    # P ≠ I: enforce that not all diagonal entries are 1
    mdl.add_constraint(mdl.sum(P[i][i] for i in range(n)) <= n - 1)

    # Enforce H M^T ≡ P H mod d
    for i in range(n):      # row index
        for l in range(k):  # column index
            # lhs = (H M^T)[i, l] = sum_j H[i,j] * M[l,j]
            lhs = mdl.sum(int(H[i, j]) * M[l][j] for j in range(k))
            # rhs = (P H)[i, l] = sum_r P[i,r] * H[r,l]
            rhs = mdl.sum(P[i][r] * int(H[r, l]) for r in range(n))

            # Modular constraint: lhs ≡ rhs mod d ⇔ lhs - rhs = d * z
            z = mdl.integer_var(name=f"Z_{i}_{l}")
            mdl.add_constraint(lhs - rhs == d * z)

    # Solve the model
    solution = mdl.solve(log_output=True)

    if not solution:
        print("No solution found.")
        return None, None

    # Extract solution values
    M_val = np.array([[int(round(solution[M[i][j]])) % d for j in range(k)] for i in range(k)])
    P_val = np.array([[int(round(solution[P[i][j]])) for j in range(n)] for i in range(n)])

    return M_val, P_val

In [ ]:
failures = []

for i in range(1):
    d = 2
    n_paulis = 10
    n_qudits = 10
    ham = random_pauli_hamiltonian(n_paulis, [d] * n_qudits, mode='uniform')
    H = ham.symplectic()

    M, P = solve_HT_equals_PH(H, d=d)

    if M is not None:
        print("Matrix M:")
        print(M)
        print("Permutation P:")
        print(P)

        # Verify: H @ M.T ≡ P @ H  (mod d)
        left = (H @ M.T) % d
        right = (P @ H) % d
        print("Verification:")
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures.append((H, M, P))

0
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 221 rows, 700 columns, and 3470 nonzeros.
Reduced MIP has 500 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.56 ticks)


Probing time = 0.00 sec. (0.84 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 221 rows, 700 columns, and 3470 nonzeros.
Reduced MIP has 500 binaries, 200 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.88 ticks)
Probing time = 0.00 sec. (0.83 ticks)
Clique table members: 20.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (7.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0        0.0000   216                      0.0000      295         
Detecting symmetries...
      0     2        0.0000    60                      0.0000      295         
Elapsed time = 0.28 sec. (185.76 ticks, tree = 0.02 MB, solutions = 0)

Performing restart 1

Repeating presolve.
Tried aggregator 1 time.
MIP Presolve modified 84

In [30]:


def solve_HT_equals_PH_scip(H, d, c=None):

    n_rows, n_cols = H.shape  # general shape
    model = scip_model("HT_equals_PH")

    # M ∈ ℤ_d^{n_cols × n_cols}
    M = [[model.addVar(vtype="I", lb=0, ub=d - 1, name=f"M_{i}_{j}") for j in range(n_cols)] for i in range(n_cols)]

    # P ∈ {0,1}^{n_rows × n_rows}
    P = [[model.addVar(vtype="B", name=f"P_{i}_{j}") for j in range(n_rows)] for i in range(n_rows)]

    # P is a permutation matrix
    for i in range(n_rows):
        model.addCons(quicksum(P[i][j] for j in range(n_rows)) == 1)
    for j in range(n_rows):
        model.addCons(quicksum(P[i][j] for i in range(n_rows)) == 1)
    model.addCons(quicksum(P[i][i] for i in range(n_rows)) <= n_rows - 1)

    # Optional group constraint
    if c is not None and np.all(c == c[0]):
        for i in range(n_rows):
            for j in range(n_rows):
                if c[i] != c[j]:
                    model.addCons(P[i][j] == 0)

    # H M^T = P H  (mod d)
    for i in range(n_rows):
        for l in range(n_cols):
            lhs = quicksum(H[i, j] * M[l][j] for j in range(n_cols))  # H[i] ⋅ M^T[:,l]
            rhs = quicksum(P[i][r] * int(H[r, l]) for r in range(n_rows))
            z = model.addVar(vtype="I", name=f"Z_{i}_{l}")
            model.addCons(lhs - rhs == d * z)

    # Solve
    model.optimize()
    if model.getStatus() != "optimal":
        print("No solution found.")
        return None, None

    M_val = np.array([[round(model.getVal(M[i][j])) % d for j in range(n_cols)] for i in range(n_cols)], dtype=int)
    P_val = np.array([[round(model.getVal(P[i][j])) for j in range(n_rows)] for i in range(n_rows)], dtype=int)
    return M_val, P_val


In [5]:
H = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 1]
])

c = np.array([0, 0, 1, 1])  # Only allow swaps within blocks 0 or 1
d = 5

M, P = solve_HT_equals_PH_scip(H, d, c)
print("M =\n", M)
print("P =\n", P)

M =
presolving:
(round 1, fast)       12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 8 clqs
(round 2, fast)       12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 8 clqs
(round 3, exhaustive) 12 del vars, 4 del conss, 0 add conss, 20 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 0 impls, 8 clqs
   (0.0s) probing cycle finished: starting next cycle
(round 4, exhaustive) 13 del vars, 4 del conss, 0 add conss, 21 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 61 impls, 36 clqs
   (0.0s) probing cycle finished: starting next cycle
(round 5, exhaustive) 14 del vars, 4 del conss, 0 add conss, 21 chg bounds, 0 chg sides, 0 chg coeffs, 9 upgd conss, 67 impls, 36 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00

In [57]:
failures = []
successes = []
failures2 = []
successes2 = []

time_solve1 = 0
time_solve2 = 0
print("Starting tests...")
for i in range(10):
    d = 2
    n_paulis = 5
    n_qudits = 5
    ham = random_pauli_hamiltonian(n_paulis, [d] * n_qudits, mode='uniform')
    H = ham.symplectic()
    coeffs = ham.weights

    print(i)

    time1 = time()
    M, P = solve_HT_equals_PH_scip(H, d, coeffs)
    time_solve1 += time() - time1
    if M is not None:
        # print("Matrix M:")
        # print(M)
        # print("Permutation P:")
        # print(P)

        # Verify: H @ M.T ≡ P @ H  (mod d)
        left = (H @ M.T) % d
        right = (P @ H) % d
        print("Verification:")
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures.append((H, M, P))
        else:
            successes.append((H, M, P))
    time2 = time()
    M, P = solve_HT_equals_PH(H, d=d)
    if M is not None:
        # print("Matrix M:")
        # print(M)
        # print("Permutation P:")
        # print(P)

        # Verify: H @ M.T ≡ P @ H  (mod d)
        left = (H @ M.T) % d
        right = (P @ H) % d
        print("Verification:")
        print(np.array_equal(left, right))
        if not np.array_equal(left, right) or np.all(M - np.eye(M.shape[0], dtype=int) == 0):
            failures2.append((H, M, P))
        else:
            successes2.append((H, M, P))
    time_solve2 += time() - time2

print(f"Total time for SCIP solver: {time_solve1:.2f} seconds")
print(f"Total time for PH2 solver: {time_solve2:.2f} seconds")

Starting tests...
0
presolving:
(round 1, fast)       10 del vars, 0 del conss, 0 add conss, 50 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
(round 2, exhaustive) 10 del vars, 0 del conss, 0 add conss, 50 chg bounds, 0 chg sides, 0 chg coeffs, 61 upgd conss, 0 impls, 10 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 24 deleted vars, 5 deleted constraints, 0 added constraints, 50 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 13 implications, 10 cliques
presolved problem has 151 variables (106 bin, 45 int, 0 impl, 0 cont) and 56 constraints
     10 constraints of type <setppc>
     45 constraints of type <xor>
      1 constraints of type <logicor>
transformed objective value is always integral (scale: 1)
Presolving T

IndexError: list index out of range

In [56]:
# print(len(failures))

i = 0
for H, M, P in successes:
    # print("H =\n", H)
    # print("M =\n", M)
    # print("P =\n", P)
    # print("Check (H @ M.T) % d == P @ H:")
    print(np.array_equal((H @ M.T) % d, P @ H))
    print(np.array_equal((H @ successes2[i][1].T) % d, successes2[i][2] @ H))
    # print(np.array_equal(M, successes2[i][1]))
    # print(M, successes2[i][1])

    # print("-" * 40)
    i += 1

# print(np.all(successes[0][0]== successes2[0][0]))
print(successes[1][2])
print(successes2[1][2])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]]
[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]
